In [46]:
import sys

import test
sys.path.insert(0, "../../hytek-parser")  # path to your forked parser folder
from hytek_parser import hy3_parser
from hytek_parser import export_xls

file_path = "../SampleMeetResults/Meet Results-PCS Spring Home Meet 2025-22Mar2025-002.hy3"

parsed_file = hy3_parser.parse_hy3(file_path)
events = parsed_file.meet.events
print(events)
events_numbers = list((events.keys()))
event = events[events_numbers[0]]
print(event)
entry = event.entries
testEntry = entry[0]
print(testEntry.swimmers[0].first_name, testEntry.swimmers[0].last_name)
print(testEntry.finals_time)

    


Invalid line code: B3
{'14': Event(number='14', distance=50, stroke=<Stroke.FREESTYLE: 'A'>, course=<Course.SCY: 'Y'>, date_=None, fee=0.0, relay=False, relay_team_id=None, relay_swim_team_code=None, gender=<Gender.MALE: 'M'>, gender_age=<GenderAge.MEN_S: 'M'>, age_min=0, age_max=99, open_=False, entries=[EventEntry(event_number='14', swimmers=[Swimmer(gender=<Gender.MALE: 'M'>, date_of_birth=datetime.date(1900, 1, 1), age=20, first_name='Jeffrey', last_name='Bzdill', nick_name='', middle_initial='', meet_id=12, team_id=12, usa_swimming_id='12345', team_code='BKNL')], relay=False, seed_time=30.0, seed_course=<Course.SCY: 'Y'>, converted_seed_time=30.0, converted_seed_time_course=<Course.SCY: 'Y'>, prelim_time=None, prelim_splits={}, prelim_course=None, prelim_time_code=None, prelim_dq_info=None, prelim_heat=None, prelim_lane=None, prelim_heat_place=None, prelim_overall_place=None, prelim_date=None, swimoff_time=None, swimoff_splits={}, swimoff_course=None, swimoff_time_code=None, swimo

In [85]:
from collections import defaultdict
from hytek_parser.hy3.enums import (
    Course,
    DisqualificationCode,
    Gender,
    GenderAge,
    ResultType,
    Stroke,
    WithTimeTimeCode,
)

def get_event_name(event):
    gender = {
        Gender.MALE: "Men's",
        Gender.FEMALE: "Women's",
        Gender.UNKNOWN: "Mixed",
    }.get(event.gender, "Unknown")

    stroke = {
        Stroke.FREESTYLE: "Freestyle",
        Stroke.BACKSTROKE: "Backstroke",
        Stroke.BREASTSTROKE: "Breaststroke",
        Stroke.BUTTERFLY: "Butterfly",
        Stroke.MEDLEY: "Medley",
    }.get(event.stroke, "Unknown")

    return f"{gender} {event.distance} {stroke} ({event.course.name})"

results = defaultdict()
for i in range(len(events_numbers)):
    event = events[events_numbers[i]]
    event_name = get_event_name(event)
    # print(event_name)
    entry = event.entries
    event_results = []
    for j in range(len(entry)):
        singleEntry = entry[j]
        swimmer_name = f"{singleEntry.swimmers[0].first_name} {singleEntry.swimmers[0].middle_initial + " " if singleEntry.swimmers[0].middle_initial else ''}{singleEntry.swimmers[0].last_name}"
        swimmer_prelim_time = singleEntry.prelim_time
        swimmer_swimmoff_time = singleEntry.swimoff_time
        swimmer_final_time = singleEntry.finals_time
        resultEntry = {
            "swimmer": swimmer_name,
            "prelim_time": swimmer_prelim_time,
            "swimoff_time": swimmer_swimmoff_time,
            "final_time": swimmer_final_time,
        }
        event_results.append(resultEntry)

    sorted_eventResults = sorted(event_results, key=lambda x: float('inf') if x['final_time'] == 0.0 or x['final_time'] is None else x['final_time'])
    results[event_name] = sorted_eventResults
    
print(results["Women's 100 Freestyle (SCY)"])

[{'swimmer': 'Sarah Tower', 'prelim_time': None, 'swimoff_time': None, 'final_time': 70.06}, {'swimmer': 'Haley Dietz', 'prelim_time': None, 'swimoff_time': None, 'final_time': 76.65}, {'swimmer': 'Maryam Atikem', 'prelim_time': None, 'swimoff_time': None, 'final_time': 82.0}, {'swimmer': 'Katherine Muth', 'prelim_time': None, 'swimoff_time': None, 'final_time': 87.66}]


In [93]:
from scipy.interpolate import interp1d
import numpy as np

pointSystem15plus = {
    "Men's 100 Freestyle (SCY)": {
        41.23: 1000,
        43.56: 950,
        45.89: 900,
        46.99: 850,
        48.09: 800,
        49.19: 750,
        50.29: 700,
        51.34: 650,
        52.49: 600,
        53.56: 550,
        54.64: 500,
        55.71: 450,
        56.79: 400,
        57.89: 350,
        58.99: 300,
        60.09: 250,
        61.19: 200,
        62.19: 150,
        63.19: 100
    },
    "Women's 100 Freestyle (SCY)": {
        46.09: 1000,
        48.69: 950,
        51.29: 900,
        52.49: 850,
        53.69: 800,
        54.89: 750,
        56.09: 700,
        57.34: 650,
        58.59: 600,
        59.81: 550,
        61.04: 500,
        62.36: 450,
        63.49: 400,
        64.69: 350,
        65.89: 300,
        67.09: 250,
        68.29: 200,
        69.29: 150,
        70.29: 100
    }
}

def get_interpolated_score(event_name: str, gender: str, time_seconds: float, points_dict: dict) -> float:
    """
    Interpolates a score from the pointSystem15plus dict based on event, gender, and performance time in seconds.
    
    Args:
        event_name (str): Event name, e.g., "100 Freestyle (SCY)"
        gender (str): 'Men' or 'Women'
        time_seconds (float): Performance time in total seconds
        points_dict (dict): Dictionary of times to scores
    
    Returns:
        float: Interpolated score
    """
    if time_seconds < 0:
        raise ValueError("Time cannot be negative")
    if time_seconds == 0:
        return 0.0
    key = f"{gender}'s {event_name}"
    if key not in points_dict:
        raise ValueError(f"No data found for event '{key}'")
    
    time_points = sorted(points_dict[key].items())  # list of (time, points)
    times = [t for t, _ in time_points]
    scores = [s for _, s in time_points]

    f = interp1d(times, scores, bounds_error=False, fill_value="extrapolate")
    score = float(f(time_seconds)) 
    return score if score > 0 else 0.0

In [90]:

for swimmer_result in results["Women's 100 Freestyle (SCY)"]:
    swimmer_name = swimmer_result["swimmer"]
    swimmer_time = swimmer_result["final_time"]
    if swimmer_time is not None or swimmer_time != 0.0:
        # Convert time to total seconds
        minutes, seconds = divmod(swimmer_time, 60)
        total_seconds = minutes * 60 + seconds
        # Get the interpolated score
        score = get_interpolated_score("100 Freestyle (SCY)", "Women", total_seconds, pointSystem15plus)
        print(f"Swimmer: {swimmer_name}, Time: {swimmer_time}, Score: {score}")

Swimmer: Sarah Tower, Time: 70.06, Score: 111.5000000000002
Swimmer: Haley Dietz, Time: 76.65, Score: 0.0
Swimmer: Maryam Atikem, Time: 82.0, Score: 0.0
Swimmer: Katherine Muth, Time: 87.66, Score: 0.0


In [94]:
for swimmer_result in results["Men's 100 Freestyle (SCY)"]:
    swimmer_name = swimmer_result["swimmer"]
    swimmer_time = swimmer_result["final_time"]
    if swimmer_time is not None or swimmer_time != 0.0:
        # Convert time to total seconds
        minutes, seconds = divmod(swimmer_time, 60)
        total_seconds = minutes * 60 + seconds
        # Get the interpolated score
        score = get_interpolated_score("100 Freestyle (SCY)", "Men", total_seconds, pointSystem15plus)
        print(f"Swimmer: {swimmer_name}, Time: {swimmer_time}, Score: {score}")

Swimmer: Maximus Forster, Time: 52.21, Score: 612.1739130434784
Swimmer: Harrison E Sly, Time: 54.05, Score: 527.314814814815
Swimmer: Michael Belch, Time: 54.93, Score: 486.44859813084116
Swimmer: Colton Park, Time: 55.38, Score: 465.42056074766344
Swimmer: Jack Moriarty, Time: 55.85, Score: 443.5185185185185
Swimmer: Evan Dabakarov, Time: 56.37, Score: 419.44444444444457
Swimmer: Luke Rutkowski, Time: 56.39, Score: 418.5185185185185
Swimmer: Jathan Krall, Time: 57.99, Score: 345.4545454545454
Swimmer: Jeffrey Bzdill, Time: 58.97, Score: 300.90909090909105
Swimmer: Seth Cooper, Time: 64.34, Score: 42.499999999999716
Swimmer: Daniel Chimes, Time: 0.0, Score: 0.0
Swimmer: Matvey Zoukovski, Time: 0.0, Score: 0.0
Swimmer: Declan Rainowski, Time: 0.0, Score: 0.0
Swimmer: Emre Oncel, Time: 0.0, Score: 0.0
